In [1]:
import os
import warnings

warnings.simplefilter(action="ignore")
os.environ["GRPC_VERBOSITY"] = "NONE"

# Prerequisites

Please make sure your environmental variables and dependencies are ready to use LLM services.

In [2]:
from dotenv import load_dotenv

load_dotenv("../../.env_api")

True

# How to input langrila's modules

In langrila, universal message system allows you to handle each client message in a same way.

# Import modules

In [3]:
from langrila import (
    InMemoryConversationMemory,
    ToolConfig,
    ToolParameter,
    ToolProperty,
)
from langrila.claude import ClaudeFunctionalChat
from langrila.gemini import GeminiFunctionalChat
from langrila.openai import OpenAIFunctionalChat

# Define tools

In this example, we can use these tools.

In [4]:
def power_disco_ball(power: bool) -> bool:
    """Powers the spinning disco ball."""
    return f"Disco ball is {'spinning!' if power else 'stopped.'}"


def start_music(energetic: bool, loud: bool, bpm: str) -> str:
    """Play some music matching the specified parameters.

    Args:
      energetic: Whether the music is energetic or not.
      loud: Whether the music is loud or not.
      bpm: The beats per minute of the music.

    Returns: The name of the song being played.
    """
    return f"Starting music! {energetic=} {loud=}, {bpm=}"


def dim_lights(brightness: float) -> bool:
    """Dim the lights.

    Args:
      brightness: The brightness of the lights, 0.0 is off, 1.0 is full.
    """
    return f"Lights are now set to {brightness}"

# Tool definition

In langrila, you can define tool configs via ToolProperty, ToolParameter and ToolConfig object.

In [5]:
tool_configs = [
    ToolConfig(
        name="power_disco_ball",
        description="Powers the spinning disco ball.",
        parameters=ToolParameter(
            properties=[
                ToolProperty(
                    name="power",
                    type="boolean",
                    description="Boolean to spin disco ball.",
                ),
            ],
            required=["power"],
        ),
    ),
    ToolConfig(
        name="start_music",
        description="Play some music matching the specified parameters.",
        parameters=ToolParameter(
            properties=[
                ToolProperty(
                    name="energetic",
                    type="boolean",
                    description="Whether the music is energetic or not.",
                ),
                ToolProperty(
                    name="loud", type="boolean", description="Whether the music is loud or not."
                ),
                ToolProperty(
                    name="bpm",
                    type="string",
                    description="The beats per minute of the music.",
                    enum=["60", "120", "180"],  # Gemini doesn't support numerical enum
                ),
            ],
            required=["energetic", "loud", "bpm"],
        ),
    ),
    ToolConfig(
        name="dim_lights",
        description="Dim the lights.",
        parameters=ToolParameter(
            properties=[
                ToolProperty(
                    name="brightness",
                    type="number",
                    description="The brightness of the lights, 0.0 is off, 1.0 is full.",
                ),
            ],
            required=["brightness"],
        ),
    ),
]

In [6]:
prompt = "Turn this place into a comfortable party mood!"

### OpenAI Chat Completion

ToolConfig object is internally converted to client tool configs

In [7]:
chat_openai = OpenAIFunctionalChat(
    api_key_env_name="API_KEY",
    model_name="gpt-4o-2024-08-06",
    tools=[power_disco_ball, start_music, dim_lights],
    tool_configs=tool_configs,
)

In [8]:
response = chat_openai.run(prompt)

# Response is CompletionResults object
response

CompletionResults(message=Message(role='assistant', content=[TextContent(text='The place is now ready for a party! The disco ball is spinning, energetic music with a 120 BPM is playing, and the lights are dimmed to create a comfortable atmosphere. Enjoy the party!')], name=None), usage=Usage(prompt_tokens=305, completion_tokens=116, total_tokens=421), prompt=[{'role': 'user', 'content': [{'type': 'text', 'text': 'Turn this place into a comfortable party mood!'}], 'name': 'User'}, {'content': None, 'refusal': None, 'role': 'assistant', 'function_call': None, 'tool_calls': [{'id': 'call_5vxFCHm9zKFqKKzOKhWiq63b', 'function': {'arguments': '{"power": true}', 'name': 'power_disco_ball'}, 'type': 'function'}, {'id': 'call_VpEFSp5ZFFX780cXxMLA8ElH', 'function': {'arguments': '{"energetic": true, "loud": false, "bpm": "120"}', 'name': 'start_music'}, 'type': 'function'}, {'id': 'call_UFT5Q46iZoOEJ0hIYwGbgFC8', 'function': {'arguments': '{"brightness": 0.5}', 'name': 'dim_lights'}, 'type': 'fu

In [9]:
response.model_dump()

{'message': {'role': 'assistant',
  'content': [{'text': 'The place is now ready for a party! The disco ball is spinning, energetic music with a 120 BPM is playing, and the lights are dimmed to create a comfortable atmosphere. Enjoy the party!'}],
  'name': None},
 'usage': {'model_name': 'gpt-4o-2024-08-06',
  'prompt_tokens': 305,
  'completion_tokens': 116},
 'prompt': [{'role': 'user',
   'content': [{'type': 'text',
     'text': 'Turn this place into a comfortable party mood!'}],
   'name': 'User'},
  {'content': None,
   'refusal': None,
   'role': 'assistant',
   'function_call': None,
   'tool_calls': [{'id': 'call_5vxFCHm9zKFqKKzOKhWiq63b',
     'function': {'arguments': '{"power": true}', 'name': 'power_disco_ball'},
     'type': 'function'},
    {'id': 'call_VpEFSp5ZFFX780cXxMLA8ElH',
     'function': {'arguments': '{"energetic": true, "loud": false, "bpm": "120"}',
      'name': 'start_music'},
     'type': 'function'},
    {'id': 'call_UFT5Q46iZoOEJ0hIYwGbgFC8',
     'func

Asynchronous generation

In [10]:
response = await chat_openai.arun(prompt)
response

CompletionResults(message=Message(role='assistant', content=[TextContent(text='The place is now in a comfortable party mood! The disco ball is spinning, energetic music is playing at 120 bpm, and the lights are dimmed to create the perfect atmosphere. Let the party begin!')], name=None), usage=Usage(prompt_tokens=305, completion_tokens=117, total_tokens=422), prompt=[{'role': 'user', 'content': [{'type': 'text', 'text': 'Turn this place into a comfortable party mood!'}], 'name': 'User'}, {'content': None, 'refusal': None, 'role': 'assistant', 'function_call': None, 'tool_calls': [{'id': 'call_UHijRuQrrwMA6f9HcIZpzSw6', 'function': {'arguments': '{"power": true}', 'name': 'power_disco_ball'}, 'type': 'function'}, {'id': 'call_PcHLWPLOpNWOgevsFXKg7RIQ', 'function': {'arguments': '{"energetic": true, "loud": true, "bpm": "120"}', 'name': 'start_music'}, 'type': 'function'}, {'id': 'call_et7J739rhbnORIql91231Yax', 'function': {'arguments': '{"brightness": 0.5}', 'name': 'dim_lights'}, 'typ

multiple generation

In [11]:
response = chat_openai.run(prompt, n_results=2)
response.message.model_dump()

{'role': 'assistant',
 'content': [{'text': "I've set the lights to a cozy level, got the disco ball spinning, and started some energetic music at 120 BPM to get the party going! Enjoy the comfortable party mood!"},
  {'text': 'The lights are now dimmed to create a cozy atmosphere, the disco ball is spinning, and energetic music is playing at 120 BPM to set the perfect party mood!'}],
 'name': None}

You can get the results from specific tool.

In [12]:
response = chat_openai.run(prompt, tool_choice="start_music")

# Show result
response.model_dump()

{'message': {'role': 'assistant',
  'content': [{'text': "I've started some upbeat, energetic music for a lively party mood! Let me know if there's anything else you need to set the perfect atmosphere!"}],
  'name': None},
 'usage': {'model_name': 'gpt-4o-2024-08-06',
  'prompt_tokens': 244,
  'completion_tokens': 45},
 'prompt': [{'role': 'user',
   'content': [{'type': 'text',
     'text': 'Turn this place into a comfortable party mood!'}],
   'name': 'User'},
  {'content': None,
   'refusal': None,
   'role': 'assistant',
   'function_call': None,
   'tool_calls': [{'id': 'call_qEmsXkY8cJUrxOB9o3o1oeN5',
     'function': {'arguments': '{"energetic":true,"loud":true,"bpm":"120"}',
      'name': 'start_music'},
     'type': 'function'}]},
  {'role': 'tool',
   'tool_call_id': 'call_qEmsXkY8cJUrxOB9o3o1oeN5',
   'name': 'start_music',
   'content': "Starting music! energetic=True loud=True, bpm='120'"}]}

If you specify tool_only option as True, you get the FunctionCallingResults object instead of CompletionResults object.

In [13]:
response = chat_openai.run(prompt, tool_only=True)
response

FunctionCallingResults(usage=Usage(prompt_tokens=171, completion_tokens=75, total_tokens=246), results=[Message(role='function', content=[ToolContent(output='Disco ball is spinning!', call_id='call_PfgBbuklYGzApAR8fpOU7WNa', args='{"power": true}', funcname='power_disco_ball')], name='power_disco_ball'), Message(role='function', content=[ToolContent(output="Starting music! energetic=True loud=True, bpm='120'", call_id='call_7LlGJ3NzJFUTDb93Nw61KbDW', args='{"energetic": true, "loud": true, "bpm": "120"}', funcname='start_music')], name='start_music'), Message(role='function', content=[ToolContent(output='Lights are now set to 0.5', call_id='call_JMcke4zDy1tuKrzrUYHUhPp9', args='{"brightness": 0.5}', funcname='dim_lights')], name='dim_lights')], calls=Message(role='function_call', content=[ToolCall(name='power_disco_ball', args='{"power": true}', call_id='call_PfgBbuklYGzApAR8fpOU7WNa'), ToolCall(name='start_music', args='{"energetic": true, "loud": true, "bpm": "120"}', call_id='call_7

In [14]:
response.model_dump()

{'usage': {'model_name': 'gpt-4o-2024-08-06',
  'prompt_tokens': 171,
  'completion_tokens': 75},
 'results': [{'role': 'function',
   'content': [{'output': 'Disco ball is spinning!',
     'call_id': 'call_PfgBbuklYGzApAR8fpOU7WNa',
     'args': '{"power": true}',
     'funcname': 'power_disco_ball'}],
   'name': 'power_disco_ball'},
  {'role': 'function',
   'content': [{'output': "Starting music! energetic=True loud=True, bpm='120'",
     'call_id': 'call_7LlGJ3NzJFUTDb93Nw61KbDW',
     'args': '{"energetic": true, "loud": true, "bpm": "120"}',
     'funcname': 'start_music'}],
   'name': 'start_music'},
  {'role': 'function',
   'content': [{'output': 'Lights are now set to 0.5',
     'call_id': 'call_JMcke4zDy1tuKrzrUYHUhPp9',
     'args': '{"brightness": 0.5}',
     'funcname': 'dim_lights'}],
   'name': 'dim_lights'}],
 'calls': {'role': 'function_call',
  'content': [{'name': 'power_disco_ball',
    'args': '{"power": true}',
    'call_id': 'call_PfgBbuklYGzApAR8fpOU7WNa'},
   

In [15]:
# Tool result messages

response.results

[Message(role='function', content=[ToolContent(output='Disco ball is spinning!', call_id='call_PfgBbuklYGzApAR8fpOU7WNa', args='{"power": true}', funcname='power_disco_ball')], name='power_disco_ball'),
 Message(role='function', content=[ToolContent(output="Starting music! energetic=True loud=True, bpm='120'", call_id='call_7LlGJ3NzJFUTDb93Nw61KbDW', args='{"energetic": true, "loud": true, "bpm": "120"}', funcname='start_music')], name='start_music'),
 Message(role='function', content=[ToolContent(output='Lights are now set to 0.5', call_id='call_JMcke4zDy1tuKrzrUYHUhPp9', args='{"brightness": 0.5}', funcname='dim_lights')], name='dim_lights')]

In [16]:
# Tool call messages

response.calls

Message(role='function_call', content=[ToolCall(name='power_disco_ball', args='{"power": true}', call_id='call_PfgBbuklYGzApAR8fpOU7WNa'), ToolCall(name='start_music', args='{"energetic": true, "loud": true, "bpm": "120"}', call_id='call_7LlGJ3NzJFUTDb93Nw61KbDW'), ToolCall(name='dim_lights', args='{"brightness": 0.5}', call_id='call_JMcke4zDy1tuKrzrUYHUhPp9')], name=None)

Streaming

In [8]:
response = chat_openai.stream(prompt)

# Response is CompletionResults object
[r for r in response]

[CompletionResults(message=Message(role='assistant', content=[TextContent(text='')], name=None), usage=Usage(prompt_tokens=0, completion_tokens=0, total_tokens=0), prompt=[{}]),
 CompletionResults(message=Message(role='assistant', content=[TextContent(text='The')], name=None), usage=Usage(prompt_tokens=0, completion_tokens=0, total_tokens=0), prompt=[{}]),
 CompletionResults(message=Message(role='assistant', content=[TextContent(text='The disco')], name=None), usage=Usage(prompt_tokens=0, completion_tokens=0, total_tokens=0), prompt=[{}]),
 CompletionResults(message=Message(role='assistant', content=[TextContent(text='The disco ball')], name=None), usage=Usage(prompt_tokens=0, completion_tokens=0, total_tokens=0), prompt=[{}]),
 CompletionResults(message=Message(role='assistant', content=[TextContent(text='The disco ball is')], name=None), usage=Usage(prompt_tokens=0, completion_tokens=0, total_tokens=0), prompt=[{}]),
 CompletionResults(message=Message(role='assistant', content=[TextC

In [11]:
response = chat_openai.astream(prompt)

# Response is CompletionResults object
[r async for r in response]

[CompletionResults(message=Message(role='assistant', content=[TextContent(text='')], name=None), usage=Usage(prompt_tokens=0, completion_tokens=0, total_tokens=0), prompt=[{}]),
 CompletionResults(message=Message(role='assistant', content=[TextContent(text='The')], name=None), usage=Usage(prompt_tokens=0, completion_tokens=0, total_tokens=0), prompt=[{}]),
 CompletionResults(message=Message(role='assistant', content=[TextContent(text='The party')], name=None), usage=Usage(prompt_tokens=0, completion_tokens=0, total_tokens=0), prompt=[{}]),
 CompletionResults(message=Message(role='assistant', content=[TextContent(text='The party mood')], name=None), usage=Usage(prompt_tokens=0, completion_tokens=0, total_tokens=0), prompt=[{}]),
 CompletionResults(message=Message(role='assistant', content=[TextContent(text='The party mood is')], name=None), usage=Usage(prompt_tokens=0, completion_tokens=0, total_tokens=0), prompt=[{}]),
 CompletionResults(message=Message(role='assistant', content=[TextC

# Pydantic tools

You can define tools as pydantic models. To use pydantic model as tool, make sure the following points: 

- Write description of top level docstring field in the class. This is used for tool description.
- Define each argument fileds with description
- Implement run() method.

Tool definition using `ToolProperty`, `ToolParameter` and `ToolConfig` is almost equivalent to the following way. 

In [17]:
from enum import Enum
from typing import Literal

from pydantic import BaseModel, Field


class PowerDiscoBall(BaseModel):
    """Powers the spinning disco ball."""

    power: bool = Field(description="Boolean to spin disco ball.")

    def run(self) -> str:
        return f"Disco ball is {'spinning!' if self.power else 'stopped.'}"


class DimLights(BaseModel):
    """Dim the lights."""

    brightness: float = Field(description="The brightness of the lights, 0.0 is off, 1.0 is full.")

    def run(self) -> str:
        return f"Lights are now set to {self.brightness}"


class BPM(Enum):
    """The beats per minute of the music."""

    SLOW = "60"
    MEDIUM = "120"
    FAST = "180"


class StartMusic(BaseModel):
    """Play some music matching the specified parameters."""

    energetic: bool = Field(description="Whether the music is energetic or not.")
    loud: bool = Field(description="Whether the music is loud or not.")
    bpm: BPM = Field(description="The beats per minute of the music.")
    # bpm: Literal["60", "120", "180"] = Field(..., description="The beats per minute of the music.") # This also works

    def run(self) -> str:
        return f"Starting music! {self.energetic=} {self.loud=}, {self.bpm=}"

In [18]:
# Convert to ToolConfig

tools = [PowerDiscoBall, StartMusic, DimLights]
tool_configs_pydantic = [ToolConfig.from_pydantic(tool) for tool in tools]

In [19]:
chat_openai = OpenAIFunctionalChat(
    api_key_env_name="API_KEY",
    model_name="gpt-4o-mini-2024-07-18",
    tools=tools,
    tool_configs=tool_configs_pydantic,
)

In [20]:
response = chat_openai.run(prompt)

# Response is CompletionResults object
response.model_dump()

{'message': {'role': 'assistant',
  'content': [{'text': 'The party mood is set! The disco ball is spinning, the energetic music is blasting at 120 BPM, and the lights are dimmed to create a cozy atmosphere. Get ready to have some fun! 🎉🕺💃'}],
  'name': None},
 'usage': {'model_name': 'gpt-4o-mini-2024-07-18',
  'prompt_tokens': 320,
  'completion_tokens': 123},
 'prompt': [{'role': 'user',
   'content': [{'type': 'text',
     'text': 'Turn this place into a comfortable party mood!'}],
   'name': 'User'},
  {'content': None,
   'refusal': None,
   'role': 'assistant',
   'function_call': None,
   'tool_calls': [{'id': 'call_bmDDVTtc45wIIACZ0wqyIIKy',
     'function': {'arguments': '{"power": true}', 'name': 'PowerDiscoBall'},
     'type': 'function'},
    {'id': 'call_BNGkWLX7H7lUWBKw28UCqA6c',
     'function': {'arguments': '{"energetic": true, "loud": true, "bpm": "120"}',
      'name': 'StartMusic'},
     'type': 'function'},
    {'id': 'call_7Xcr2zMu2bXJ6gnsZOAmn6qZ',
     'functio

### Array property for OpenAI API

Function Calling with array property for OpenAI Chat Completion.

In [21]:
def get_ingredients(ingredients: list[str]) -> str:
    """Prepare ingredients to make a meal."""
    return f"Preparing {ingredients}."


tool_configs_array = [
    ToolConfig(
        name="get_ingredients",
        description="Prepare ingredients to make a meal.",
        parameters=ToolParameter(
            properties=[
                ToolProperty(
                    name="ingredients",
                    type="array",
                    description="List of ingredients.",
                    items={"type": "string"},
                ),
            ],
            required=["ingredients"],
        ),
    ),
]

In [22]:
chat_openai = OpenAIFunctionalChat(
    api_key_env_name="API_KEY",
    model_name="gpt-4o-2024-08-06",
    tools=[get_ingredients],
    tool_configs=tool_configs_array,
)

In [23]:
response = chat_openai.run(
    prompt="What are the ingredients to make omelet rice?",
    tool_choice="get_ingredients",
)
response.model_dump()

{'message': {'role': 'assistant',
  'content': [{'text': 'To make omelet rice, you will need the following ingredients:\n\n- Eggs\n- Cooked rice\n- Chicken (optional, can substitute with other proteins or make it vegetarian)\n- Onion\n- Bell pepper\n- Ketchup\n- Soy sauce\n- Salt\n- Pepper\n- Butter\n- Vegetable oil\n\nThese ingredients are typically used to create the savory filling and the fluffy omelet that wraps around the rice mixture.'}],
  'name': None},
 'usage': {'model_name': 'gpt-4o-2024-08-06',
  'prompt_tokens': 194,
  'completion_tokens': 124},
 'prompt': [{'role': 'user',
   'content': [{'type': 'text',
     'text': 'What are the ingredients to make omelet rice?'}],
   'name': 'User'},
  {'content': None,
   'refusal': None,
   'role': 'assistant',
   'function_call': None,
   'tool_calls': [{'id': 'call_acT2l5IZ2xGqaxjnM7CemiKd',
     'function': {'arguments': '{"ingredients":["eggs","cooked rice","chicken","onion","bell pepper","ketchup","soy sauce","salt","pepper","bu

### Azure OpenAI Chat Completion 

In [24]:
chat_openai_azure = OpenAIFunctionalChat(
    api_key_env_name="AZURE_API_KEY",
    model_name="gpt-4o-2024-05-13",
    api_type="azure",
    api_version="2024-05-01-preview",
    endpoint_env_name="AZURE_ENDPOINT",
    deployment_id_env_name="AZURE_DEPLOYMENT_ID",
    tools=[power_disco_ball, start_music, dim_lights],
    tool_configs=tool_configs,
)

In [25]:
response = chat_openai_azure.run(prompt)

response.model_dump()

{'message': {'role': 'assistant',
  'content': [{'text': 'The lights are dimmed to a cozy level, the disco ball is spinning, and some energetic music is playing at just the right loudness. The party mood is set—enjoy!'}],
  'name': None},
 'usage': {'model_name': 'gpt-4o-2024-05-13',
  'prompt_tokens': 306,
  'completion_tokens': 113},
 'prompt': [{'role': 'user',
   'content': [{'type': 'text',
     'text': 'Turn this place into a comfortable party mood!'}],
   'name': 'User'},
  {'content': None,
   'refusal': None,
   'role': 'assistant',
   'function_call': None,
   'tool_calls': [{'id': 'call_fZQKWLBWTpor17WWWcE4Le1S',
     'function': {'arguments': '{"brightness": 0.5}', 'name': 'dim_lights'},
     'type': 'function'},
    {'id': 'call_kgF9Piuuj4UzHNoz3M8m392v',
     'function': {'arguments': '{"power": true}', 'name': 'power_disco_ball'},
     'type': 'function'},
    {'id': 'call_kuoldq09Gaar84RSXQDg5FYC',
     'function': {'arguments': '{"energetic": true, "loud": true, "bpm":

In [26]:
response = await chat_openai_azure.arun(prompt)

response.model_dump()

{'message': {'role': 'assistant',
  'content': [{'text': "I've turned on some energetic music (not too loud, perfect for mingling) at 120 BPM, dimmed the lights to 50%, and got the disco ball spinning. Let's get this party started! 🎉"}],
  'name': None},
 'usage': {'model_name': 'gpt-4o-2024-05-13',
  'prompt_tokens': 306,
  'completion_tokens': 119},
 'prompt': [{'role': 'user',
   'content': [{'type': 'text',
     'text': 'Turn this place into a comfortable party mood!'}],
   'name': 'User'},
  {'content': None,
   'refusal': None,
   'role': 'assistant',
   'function_call': None,
   'tool_calls': [{'id': 'call_gem6WwaJK50ixNQzh3USWvIS',
     'function': {'arguments': '{"energetic": true, "loud": false, "bpm": "120"}',
      'name': 'start_music'},
     'type': 'function'},
    {'id': 'call_AcsdjO1N6UKiY9TVDWLnbZ2y',
     'function': {'arguments': '{"brightness": 0.5}', 'name': 'dim_lights'},
     'type': 'function'},
    {'id': 'call_zzm1cfaUSx1DLhFgsBuxTacD',
     'function': {'arg

In [27]:
response = chat_openai_azure.run(prompt, tool_choice="start_music")

response.model_dump()

{'message': {'role': 'assistant',
  'content': [{'text': "Music is on! Let's elevate the ambiance with some cool lighting. Dim the main lights and switch on some colorful LED bulbs or fairy lights. Set up a few cozy seating areas with cushions and lounge chairs. Place some snacks and drinks on a table, and don't forget to keep the dance floor clear! Enjoy the party!"}],
  'name': None},
 'usage': {'model_name': 'gpt-4o-2024-05-13',
  'prompt_tokens': 244,
  'completion_tokens': 81},
 'prompt': [{'role': 'user',
   'content': [{'type': 'text',
     'text': 'Turn this place into a comfortable party mood!'}],
   'name': 'User'},
  {'content': None,
   'refusal': None,
   'role': 'assistant',
   'function_call': None,
   'tool_calls': [{'id': 'call_oVobkzh6ea3mVSg2C9c9Olbt',
     'function': {'arguments': '{"energetic":true,"loud":true,"bpm":"120"}',
      'name': 'start_music'},
     'type': 'function'}]},
  {'role': 'tool',
   'tool_call_id': 'call_oVobkzh6ea3mVSg2C9c9Olbt',
   'name': '

### Gemini on Google Generative AI

You can use tools for Gemini as well which defined for OpenAI Chat Completion.

In [28]:
gemini = GeminiFunctionalChat(
    api_key_env_name="GEMINI_API_KEY",
    model_name="gemini-1.5-pro",
    tools=[power_disco_ball, start_music, dim_lights],
    tool_configs=tool_configs,
)

In [29]:
response = gemini.run(prompt)

# show result
response.model_dump()

{'message': {'role': 'assistant',
  'content': [{'text': "Alright, I've got the disco ball spinning, the music pumping, and the lights dimmed! Let's get this party started! 🎉 \n"}],
  'name': None},
 'usage': {'model_name': 'gemini-1.5-pro',
  'prompt_tokens': 353,
  'completion_tokens': 87},
 'prompt': [parts {
    text: "Turn this place into a comfortable party mood!"
  }
  role: "user",
  parts {
    function_call {
      name: "power_disco_ball"
      args {
        fields {
          key: "power"
          value {
            bool_value: true
          }
        }
      }
    }
  }
  parts {
    function_call {
      name: "start_music"
      args {
        fields {
          key: "loud"
          value {
            bool_value: true
          }
        }
        fields {
          key: "energetic"
          value {
            bool_value: true
          }
        }
        fields {
          key: "bpm"
          value {
            string_value: "120"
          }
        }
      

Asynchronous generation

In [30]:
response = await gemini.arun(prompt)

# show result
response.model_dump()

{'message': {'role': 'assistant',
  'content': [{'text': "Alright, I've dimmed the lights, turned on the disco ball, and started some upbeat music. Let's get this party started! 🎉 \n"}],
  'name': None},
 'usage': {'model_name': 'gemini-1.5-pro',
  'prompt_tokens': 353,
  'completion_tokens': 88},
 'prompt': [parts {
    text: "Turn this place into a comfortable party mood!"
  }
  role: "user",
  parts {
    function_call {
      name: "dim_lights"
      args {
        fields {
          key: "brightness"
          value {
            number_value: 0.5
          }
        }
      }
    }
  }
  parts {
    function_call {
      name: "power_disco_ball"
      args {
        fields {
          key: "power"
          value {
            bool_value: true
          }
        }
      }
    }
  }
  parts {
    function_call {
      name: "start_music"
      args {
        fields {
          key: "loud"
          value {
            bool_value: true
          }
        }
        fields {
      

Gemini allows you to call specific tool as well as OpenAI Chat Completion.

In [31]:
response = gemini.run(prompt, tool_choice="start_music")

# show result
response.model_dump()

{'message': {'role': 'assistant',
  'content': [{'text': "Okay, I'm setting the mood!  🎶 Bumping tunes incoming! What else can I do to make this party awesome? 🎉  Need lighting? Snacks? Games? Let's go! 🎊 \n"}],
  'name': None},
 'usage': {'model_name': 'gemini-1.5-pro',
  'prompt_tokens': 276,
  'completion_tokens': 68},
 'prompt': [parts {
    text: "Turn this place into a comfortable party mood!"
  }
  role: "user",
  parts {
    function_call {
      name: "start_music"
      args {
        fields {
          key: "loud"
          value {
            bool_value: true
          }
        }
        fields {
          key: "energetic"
          value {
            bool_value: true
          }
        }
        fields {
          key: "bpm"
          value {
            string_value: "120"
          }
        }
      }
    }
  }
  role: "model",
  parts {
    function_response {
      name: "start_music"
      response {
        fields {
          key: "content"
          value {
     

Also pydantic tools is acceptable.

In [32]:
gemini = GeminiFunctionalChat(
    api_key_env_name="GEMINI_API_KEY",
    model_name="gemini-1.5-pro",
    tools=[PowerDiscoBall, StartMusic, DimLights],
    tool_configs=tool_configs_pydantic,
)

In [33]:
response = gemini.run(prompt)
response.model_dump()

{'message': {'role': 'assistant',
  'content': [{'text': "Alright, I've dimmed the lights, cranked up some energetic tunes, and got the disco ball spinning! Let's get this party started! 🎉 \n"}],
  'name': None},
 'usage': {'model_name': 'gemini-1.5-pro',
  'prompt_tokens': 353,
  'completion_tokens': 86},
 'prompt': [parts {
    text: "Turn this place into a comfortable party mood!"
  }
  role: "user",
  parts {
    function_call {
      name: "DimLights"
      args {
        fields {
          key: "brightness"
          value {
            number_value: 0.1
          }
        }
      }
    }
  }
  parts {
    function_call {
      name: "StartMusic"
      args {
        fields {
          key: "loud"
          value {
            bool_value: true
          }
        }
        fields {
          key: "energetic"
          value {
            bool_value: true
          }
        }
        fields {
          key: "bpm"
          value {
            string_value: "120"
          }
    

### Gemini on VertexAI

In [34]:
gemini_vertexai = GeminiFunctionalChat(
    model_name="gemini-1.5-flash",
    api_type="vertexai",
    project_id_env_name="PROJECT_ID",
    location_env_name="LOCATION",
    tools=[power_disco_ball, start_music, dim_lights],
    tool_configs=tool_configs,
)

In [35]:
response = gemini_vertexai.run(prompt=prompt)

# show result
response.model_dump()

{'message': {'role': 'assistant',
  'content': [{'text': "Okay, the lights are now dimmed to create a cozy atmosphere. The music is playing, with an energetic beat that's perfect for dancing, but not too loud to drown out conversation. And the disco ball is spinning, adding a touch of fun and sparkle.  \n\nIs there anything else I can do to help set the mood? Maybe some scented candles or a few more decorations?  Let me know! \n"}],
  'name': None},
 'usage': {'model_name': 'gemini-1.5-flash',
  'prompt_tokens': 181,
  'completion_tokens': 105},
 'prompt': [role: "user"
  parts {
    text: "Turn this place into a comfortable party mood!"
  },
  role: "model"
  parts {
    function_call {
      name: "dim_lights"
      args {
        fields {
          key: "brightness"
          value {
            number_value: 0.5
          }
        }
      }
    }
  }
  parts {
    function_call {
      name: "start_music"
      args {
        fields {
          key: "loud"
          value {
      

multiple generation

In [36]:
response = gemini_vertexai.run(prompt=prompt, n_results=2)
response.message.model_dump()

{'role': 'assistant',
 'content': [{'text': "Okay, I've started the music! It's got a good beat and isn't too loud, perfect for getting the party going. I've also dimmed the lights a bit and turned on the disco ball, which should add a fun, festive atmosphere. \n\nIs there anything else you'd like me to do to set the mood? Maybe I can help you with:\n\n* **Setting up some snacks and drinks?**\n* **Turning on some fun party games?**\n* **Finding a playlist with everyone's favorite songs?**\n\nJust let me know what you have in mind and I'll do my best! \n"},
  {'text': "Okay, I've started the music! It's got a good beat and isn't too loud, perfect for getting the party going. I've also dimmed the lights a bit and turned on the disco ball, which should add a fun, festive atmosphere. \n\nIs there anything else you'd like me to do to set the mood? Maybe I can help you with:\n\n* **Setting up some snacks and drinks?**\n* **Turning on some fun party games?**\n* **Finding a playlist with every

Pydantic tool use

In [37]:
gemini_vertexai = GeminiFunctionalChat(
    model_name="gemini-1.5-flash",
    api_type="vertexai",
    project_id_env_name="PROJECT_ID",
    location_env_name="LOCATION",
    tools=[PowerDiscoBall, StartMusic, DimLights],
    tool_configs=tool_configs_pydantic,
)

In [38]:
response = gemini_vertexai.run(prompt=prompt)
response.model_dump()

{'message': {'role': 'assistant',
  'content': [{'text': "Okay, I'm turning up the fun!  I've started some upbeat music with a medium tempo, dimmed the lights to a cozy level, and fired up the disco ball. \n\nLet's get this party started! \U0001faa9🎶✨ \n"}],
  'name': None},
 'usage': {'model_name': 'gemini-1.5-flash',
  'prompt_tokens': 181,
  'completion_tokens': 70},
 'prompt': [role: "user"
  parts {
    text: "Turn this place into a comfortable party mood!"
  },
  role: "model"
  parts {
    function_call {
      name: "StartMusic"
      args {
        fields {
          key: "loud"
          value {
            bool_value: false
          }
        }
        fields {
          key: "energetic"
          value {
            bool_value: true
          }
        }
        fields {
          key: "bpm"
          value {
            string_value: "120"
          }
        }
      }
    }
  }
  parts {
    function_call {
      name: "DimLights"
      args {
        fields {
          

### Anthropic Claude

tool_configs can be shared with Claude.

In [39]:
claude = ClaudeFunctionalChat(
    model_name="claude-3-5-sonnet-20240620",
    api_key_env_name="ANTHROPIC_API_KEY",
    tools=[power_disco_ball, start_music, dim_lights],
    tool_configs=tool_configs,
)

In [40]:
response = claude.run(prompt=prompt)

response.model_dump()

{'message': {'role': 'assistant',
  'content': [{'text': "Great! I've set up a comfortable party mood for you. Here's what I've done:\n\n1. Dimmed the lights to 40% brightness (0.4), which creates a cozy and inviting atmosphere without being too dark.\n2. Started some music that is energetic but not too loud, with a moderate tempo of 120 beats per minute. This should provide a good background for conversation and light dancing if desired.\n3. Turned on the disco ball to add a fun, party-like element to the room with its spinning lights.\n\nThis combination should create a comfortable party mood that's perfect for socializing and enjoying the evening. The lighting is low enough to be relaxing but not too dark, the music is upbeat without being overwhelming, and the disco ball adds a festive touch.\n\nIs there anything else you'd like me to adjust to make the party mood even more comfortable or suited to your preferences?"}],
  'name': None},
 'usage': {'model_name': 'claude-3-5-sonnet-2

Asynchronous generation

In [41]:
response = await claude.arun(prompt=prompt)

response.model_dump()

{'message': {'role': 'assistant',
  'content': [{'text': "Great! I've set up a comfortable party mood for you. Here's what I've done:\n\n1. Turned on the disco ball to add some fun, dynamic lighting to the space.\n2. Started some music that's energetic but not too loud, with a moderate tempo of 120 BPM. This should create a lively atmosphere without being overwhelming.\n3. Dimmed the lights to 60% brightness, which should provide a cozy, party-like ambiance without making it too dark.\n\nThese settings should create a comfortable party mood that's lively enough for socializing and having fun, but not too intense. The combination of the spinning disco ball, energetic music at a reasonable volume, and slightly dimmed lights should give you the perfect party atmosphere.\n\nIs there anything else you'd like me to adjust to make the mood even more comfortable or party-like?"}],
  'name': None},
 'usage': {'model_name': 'claude-3-5-sonnet-20240620',
  'prompt_tokens': 1526,
  'completion_tok

Pydantic tool use

In [42]:
claude = ClaudeFunctionalChat(
    model_name="claude-3-5-sonnet-20240620",
    api_key_env_name="ANTHROPIC_API_KEY",
    tools=[PowerDiscoBall, StartMusic, DimLights],
    tool_configs=tool_configs_pydantic,
)

In [43]:
response = claude.run(prompt=prompt)
response.model_dump()

{'message': {'role': 'assistant',
  'content': [{'text': "Great! I've set up a comfortable party mood for you. Here's what I've done:\n\n1. Dimmed the lights to 40% brightness (0.4), which should create a cozy and inviting atmosphere without being too dark.\n2. Started playing some energetic music at a moderate volume. The beats per minute (BPM) is set to 120, which is a good pace for a comfortable party – not too slow, but not too fast either.\n3. Powered on the disco ball, which will add a fun, party-like ambiance to the room with its spinning lights.\n\nThis combination should create a comfortable party mood that's perfect for socializing and having a good time. The lighting is low enough to feel relaxed but not too dark, the music is upbeat but not overpowering, and the disco ball adds a festive touch.\n\nIs there anything else you'd like to adjust? For example, if you want the lights brighter or dimmer, the music louder or quieter, or if you'd like to turn off the disco ball, just

### Claude on Amazon Bedrock

In [44]:
claude_bedrock = ClaudeFunctionalChat(
    model_name="anthropic.claude-3-sonnet-20240229-v1:0",
    api_type="bedrock",
    aws_access_key_env_name="AWS_ACCESS_KEY",
    aws_secret_key_env_name="AWS_SECRET_KEY",
    aws_region_env_name="AWS_REGION",
    tools=[power_disco_ball, start_music, dim_lights],
    tool_configs=tool_configs,
)

In [45]:
response = claude.run(prompt=prompt)

response.model_dump()

{'message': {'role': 'assistant',
  'content': [{'text': "Great! I've set up a comfortable party mood for you. Here's what I've done:\n\n1. Dimmed the lights to 40% brightness (0.4), which should create a cozy and inviting atmosphere without being too dark.\n2. Started some energetic music at a medium tempo (120 BPM) that isn't too loud. This should provide a good backdrop for conversation and light dancing if desired.\n3. Turned on the disco ball to add a fun, party-like element to the room without being overwhelming.\n\nThese settings should create a comfortable party mood that's perfect for socializing and enjoying the atmosphere. The dimmed lights and the soft glow from the disco ball will create a warm ambiance, while the energetic but not too loud music will keep the energy up without hindering conversation.\n\nIs there anything else you'd like to adjust to make the party mood even more comfortable? For example, we could change the music tempo or adjust the lighting if you prefer

In [46]:
repsonse = await claude.arun(prompt=prompt)
response.model_dump()

{'message': {'role': 'assistant',
  'content': [{'text': "Great! I've set up a comfortable party mood for you. Here's what I've done:\n\n1. Dimmed the lights to 40% brightness (0.4), which should create a cozy and inviting atmosphere without being too dark.\n2. Started some energetic music at a medium tempo (120 BPM) that isn't too loud. This should provide a good backdrop for conversation and light dancing if desired.\n3. Turned on the disco ball to add a fun, party-like element to the room without being overwhelming.\n\nThese settings should create a comfortable party mood that's perfect for socializing and enjoying the atmosphere. The dimmed lights and the soft glow from the disco ball will create a warm ambiance, while the energetic but not too loud music will keep the energy up without hindering conversation.\n\nIs there anything else you'd like to adjust to make the party mood even more comfortable? For example, we could change the music tempo or adjust the lighting if you prefer

Pydantic tool use

In [47]:
claude_bedrock = ClaudeFunctionalChat(
    model_name="anthropic.claude-3-sonnet-20240229-v1:0",
    api_type="bedrock",
    aws_access_key_env_name="AWS_ACCESS_KEY",
    aws_secret_key_env_name="AWS_SECRET_KEY",
    aws_region_env_name="AWS_REGION",
    tools=[PowerDiscoBall, StartMusic, DimLights],
    tool_configs=tool_configs_pydantic,
)

In [48]:
response = claude.run(prompt=prompt)
response.model_dump()

{'message': {'role': 'assistant',
  'content': [{'text': "Great! I've set up a comfortable party mood for you. Here's what I've done:\n\n1. Dimmed the lights: I've set the brightness to 0.5, which is a nice middle ground that creates a cozy atmosphere without being too dark.\n\n2. Started the music: I've chosen energetic music to keep the party vibe going, but set it to a lower volume (not loud) so people can still comfortably chat. The beats per minute (BPM) is set to 120, which is a moderate tempo that's great for a comfortable party mood.\n\n3. Activated the disco ball: The spinning disco ball will add a fun, party element to the room without being too overwhelming.\n\nThese settings should create a perfect balance for a comfortable party atmosphere. The dimmed lights and spinning disco ball will create a festive ambiance, while the energetic but not-too-loud music will keep the mood upbeat without hindering conversation.\n\nIs there anything else you'd like to adjust to make the pa

# Universal tool config system

Tool config the user defines is common for various llm. It is converted to client tools in clinet's FunctionCallingModule before API call. Let me show you the process of conversion.

In [49]:
from langrila.openai import OpenAIToolConfig

converted_configs = OpenAIToolConfig.from_universal_configs(tool_configs)
converted_configs

[OpenAIToolConfig(name='power_disco_ball', description='Powers the spinning disco ball.', parameters=OpenAIToolParameter(type='object', properties=[OpenAIToolProperty(name='power', type='boolean', description='Boolean to spin disco ball.', enum=None, items=None)], required=['power']), strict=None, type='function'),
 OpenAIToolConfig(name='start_music', description='Play some music matching the specified parameters.', parameters=OpenAIToolParameter(type='object', properties=[OpenAIToolProperty(name='energetic', type='boolean', description='Whether the music is energetic or not.', enum=None, items=None), OpenAIToolProperty(name='loud', type='boolean', description='Whether the music is loud or not.', enum=None, items=None), OpenAIToolProperty(name='bpm', type='string', description='The beats per minute of the music.', enum=['60', '120', '180'], items=None)], required=['energetic', 'loud', 'bpm']), strict=None, type='function'),
 OpenAIToolConfig(name='dim_lights', description='Dim the lig

Then these client tool config objects is transformed to client tools by format() method.

In [50]:
converted_configs[0].format()

{'type': 'function',
 'function': {'name': 'power_disco_ball',
  'description': 'Powers the spinning disco ball.',
  'parameters': {'type': 'object',
   'required': ['power'],
   'properties': {'power': {'type': 'boolean',
     'description': 'Boolean to spin disco ball.'}}}}}

Other client like Gemini and Claude is the same interface, so you could reuse tool_configs you initially defines.

In [51]:
from langrila.gemini.genai import GeminiToolConfig

In [52]:
converted_configs = GeminiToolConfig.from_universal_configs(tool_configs)
converted_configs

[GeminiToolConfig(name='power_disco_ball', description='Powers the spinning disco ball.', parameters=GeminiToolParameter(type='object', properties=[GeminiToolProperty(name='power', type='boolean', description='Boolean to spin disco ball.', enum=None, items=None)], required=['power']), strict=None),
 GeminiToolConfig(name='start_music', description='Play some music matching the specified parameters.', parameters=GeminiToolParameter(type='object', properties=[GeminiToolProperty(name='energetic', type='boolean', description='Whether the music is energetic or not.', enum=None, items=None), GeminiToolProperty(name='loud', type='boolean', description='Whether the music is loud or not.', enum=None, items=None), GeminiToolProperty(name='bpm', type='string', description='The beats per minute of the music.', enum=['60', '120', '180'], items=None)], required=['energetic', 'loud', 'bpm']), strict=None),
 GeminiToolConfig(name='dim_lights', description='Dim the lights.', parameters=GeminiToolParame

In [53]:
converted_configs[0].format()

name: "power_disco_ball"
description: "Powers the spinning disco ball."
parameters {
  type_: OBJECT
  properties {
    key: "power"
    value {
      type_: BOOLEAN
      description: "Boolean to spin disco ball."
    }
  }
  required: "power"
}

# Multi-turn conversation with tools

### For OpenAI Chat Completion

In [54]:
chat_openai = OpenAIFunctionalChat(
    api_key_env_name="API_KEY",
    model_name="gpt-4o-mini-2024-07-18",
    tools=[power_disco_ball, start_music, dim_lights],
    tool_configs=tool_configs,
    conversation_memory=InMemoryConversationMemory(),
    # conversation_memory=JSONConversationMemory("./test.json"), # for serialization
)

In [55]:
prompt = "Turn this place into a comfortable party mood!"

response = chat_openai.run(prompt=prompt)

# Show result
response.model_dump()

{'message': {'role': 'assistant',
  'content': [{'text': "The party mood is on! The disco ball is spinning, energetic music is playing at a lively 120 BPM, and the lights are dimmed to create a cozy atmosphere. Let's get the celebration started! 🎉🥳"}],
  'name': None},
 'usage': {'model_name': 'gpt-4o-mini-2024-07-18',
  'prompt_tokens': 305,
  'completion_tokens': 120},
 'prompt': [{'role': 'user',
   'content': [{'type': 'text',
     'text': 'Turn this place into a comfortable party mood!'}],
   'name': 'User'},
  {'content': None,
   'refusal': None,
   'role': 'assistant',
   'function_call': None,
   'tool_calls': [{'id': 'call_BDZ3yaJYtsvgtHDgGTwx1XSe',
     'function': {'arguments': '{"power": true}', 'name': 'power_disco_ball'},
     'type': 'function'},
    {'id': 'call_QL7C2MdUZvLqYqJ3ab77eq7Q',
     'function': {'arguments': '{"energetic": true, "loud": true, "bpm": "120"}',
      'name': 'start_music'},
     'type': 'function'},
    {'id': 'call_kbn0HutNCjLgpZC2KPGs6TQH',
 

Next turn

In [56]:
prompt = "Can you make bpm up more?"

response = chat_openai.run(prompt=prompt)

response.model_dump()

{'message': {'role': 'assistant',
  'content': [{'text': 'The BPM has been increased to a lively 180! The energy is definitely on the rise now. Enjoy the party! 🎶💃🕺'}],
  'name': None},
 'usage': {'model_name': 'gpt-4o-mini-2024-07-18',
  'prompt_tokens': 595,
  'completion_tokens': 57},
 'prompt': [{'role': 'user',
   'content': [{'type': 'text',
     'text': 'Turn this place into a comfortable party mood!'}],
   'name': 'User'},
  {'content': None,
   'refusal': None,
   'role': 'assistant',
   'function_call': None,
   'tool_calls': [{'id': 'call_BDZ3yaJYtsvgtHDgGTwx1XSe',
     'function': {'arguments': '{"power": true}', 'name': 'power_disco_ball'},
     'type': 'function'},
    {'id': 'call_QL7C2MdUZvLqYqJ3ab77eq7Q',
     'function': {'arguments': '{"energetic": true, "loud": true, "bpm": "120"}',
      'name': 'start_music'},
     'type': 'function'},
    {'id': 'call_kbn0HutNCjLgpZC2KPGs6TQH',
     'function': {'arguments': '{"brightness": 0.5}', 'name': 'dim_lights'},
     'typ

### For Gemini

In [57]:
gemini = GeminiFunctionalChat(
    api_key_env_name="GEMINI_API_KEY",
    model_name="gemini-1.5-pro",
    tools=[power_disco_ball, start_music, dim_lights],
    tool_configs=tool_configs,
    conversation_memory=InMemoryConversationMemory(),
    # conversation_memory=JSONConversationMemory("./test.json"), # for serialization
)

In [58]:
prompt = "Turn this place into a comfortable party mood!"

response = gemini.run(prompt=prompt)

# Show result
response.model_dump()

{'message': {'role': 'assistant',
  'content': [{'text': "Alright, I've dimmed the lights, turned on the disco ball, and started some upbeat music! Let's get this party started! 🎉 \n"}],
  'name': None},
 'usage': {'model_name': 'gemini-1.5-pro',
  'prompt_tokens': 353,
  'completion_tokens': 88},
 'prompt': [parts {
    text: "Turn this place into a comfortable party mood!"
  }
  role: "user",
  parts {
    function_call {
      name: "dim_lights"
      args {
        fields {
          key: "brightness"
          value {
            number_value: 0.5
          }
        }
      }
    }
  }
  parts {
    function_call {
      name: "power_disco_ball"
      args {
        fields {
          key: "power"
          value {
            bool_value: true
          }
        }
      }
    }
  }
  parts {
    function_call {
      name: "start_music"
      args {
        fields {
          key: "loud"
          value {
            bool_value: true
          }
        }
        fields {
      

Tool calling is not needed additionally if the thing is already known.

In [59]:
prompt = "What is the current bpm?"

response = gemini.run(prompt=prompt)

response.model_dump()

{'message': {'role': 'assistant',
  'content': [{'text': 'The current music is set to 120 bpm.  Do you want me to change it? 🎶 \n'}],
  'name': None},
 'usage': {'model_name': 'gemini-1.5-pro',
  'prompt_tokens': 185,
  'completion_tokens': 23},
 'prompt': [parts {
    text: "Turn this place into a comfortable party mood!"
  }
  role: "user",
  parts {
    function_call {
      name: "dim_lights"
      args {
        fields {
          key: "brightness"
          value {
            number_value: 0.5
          }
        }
      }
    }
  }
  parts {
    function_call {
      name: "power_disco_ball"
      args {
        fields {
          key: "power"
          value {
            bool_value: true
          }
        }
      }
    }
  }
  parts {
    function_call {
      name: "start_music"
      args {
        fields {
          key: "loud"
          value {
            bool_value: true
          }
        }
        fields {
          key: "energetic"
          value {
            bo

## Multi-turn conversation using tools with multiple model

The universal message and tool system allows us to chat multiple model across client.

In [60]:
from langrila import InMemoryConversationMemory

shared_memory = InMemoryConversationMemory()

chat_openai = OpenAIFunctionalChat(
    api_key_env_name="API_KEY",
    model_name="gpt-4o-mini-2024-07-18",
    tools=[power_disco_ball, start_music, dim_lights],
    tool_configs=tool_configs,
    conversation_memory=shared_memory,
)

gemini = GeminiFunctionalChat(
    api_key_env_name="GEMINI_API_KEY",
    model_name="gemini-1.5-pro",
    tools=[power_disco_ball, start_music, dim_lights],
    tool_configs=tool_configs,
    conversation_memory=shared_memory,
)

claude = ClaudeFunctionalChat(
    model_name="claude-3-5-sonnet-20240620",
    api_key_env_name="ANTHROPIC_API_KEY",
    tools=[power_disco_ball, start_music, dim_lights],
    tool_configs=tool_configs,
    conversation_memory=shared_memory,
)

In [61]:
prompt = "Turn this place into a comfortable party mood!"

response = chat_openai.run(prompt)

response.model_dump()

{'message': {'role': 'assistant',
  'content': [{'text': 'The party mood is all set! The disco ball is spinning, energetic music is playing at a lively beat of 120 BPM, and the lights are dimmed to create a cozy atmosphere. Get ready to have an amazing time! 🎉💃🕺'}],
  'name': None},
 'usage': {'model_name': 'gpt-4o-mini-2024-07-18',
  'prompt_tokens': 305,
  'completion_tokens': 128},
 'prompt': [{'role': 'user',
   'content': [{'type': 'text',
     'text': 'Turn this place into a comfortable party mood!'}],
   'name': 'User'},
  {'content': None,
   'refusal': None,
   'role': 'assistant',
   'function_call': None,
   'tool_calls': [{'id': 'call_2meoHGpU9xnEskOBHYvOUVHa',
     'function': {'arguments': '{"power": true}', 'name': 'power_disco_ball'},
     'type': 'function'},
    {'id': 'call_eS3k3cRgJzCjIdQH5Q1HsSsB',
     'function': {'arguments': '{"energetic": true, "loud": true, "bpm": "120"}',
      'name': 'start_music'},
     'type': 'function'},
    {'id': 'call_coFf4aSVXyk4M4

Claude can continue conversation without any additional process.

In [62]:
prompt = "Can you make bpm up more?"

response = claude.run(prompt=prompt)

response.model_dump()

{'message': {'role': 'assistant',
  'content': [{'text': "Great! I've increased the music's BPM to 180. The music is now playing at a faster tempo, which should create an even more energetic and lively party atmosphere. The music remains energetic and loud, perfect for dancing and keeping the party spirit high.\n\nIs there anything else you'd like to adjust to enhance the party mood?"}],
  'name': None},
 'usage': {'model_name': 'claude-3-5-sonnet-20240620',
  'prompt_tokens': 2091,
  'completion_tokens': 242},
 'prompt': [{'role': 'user',
   'content': [{'text': 'Turn this place into a comfortable party mood!',
     'type': 'text'}]},
  {'role': 'assistant',
   'content': [{'id': 'toolu_2meoHGpU9xnEskOBHYvOUVHa',
     'type': 'tool_use',
     'input': {'power': True},
     'name': 'power_disco_ball'},
    {'id': 'toolu_eS3k3cRgJzCjIdQH5Q1HsSsB',
     'type': 'tool_use',
     'input': {'energetic': True, 'loud': True, 'bpm': '120'},
     'name': 'start_music'},
    {'id': 'toolu_coFf4a

Go back to OpenAI Chat Completion

In [63]:
prompt = "Please turn music more relax."

response = chat_openai.run(prompt)

response.model_dump()

{'message': {'role': 'assistant',
  'content': [{'text': "I've adjusted the music to a more relaxing vibe with a BPM of 60. The volume is also lowered for a calm atmosphere, perfect for winding down or chatting with friends. Let me know if you'd like to make any further changes!"}],
  'name': None},
 'usage': {'model_name': 'gpt-4o-mini-2024-07-18',
  'prompt_tokens': 875,
  'completion_tokens': 73},
 'prompt': [{'role': 'user',
   'content': [{'type': 'text',
     'text': 'Turn this place into a comfortable party mood!'}],
   'name': 'User'},
  {'content': None,
   'refusal': None,
   'role': 'assistant',
   'function_call': None,
   'tool_calls': [{'id': 'call_2meoHGpU9xnEskOBHYvOUVHa',
     'function': {'arguments': '{"power": true}', 'name': 'power_disco_ball'},
     'type': 'function'},
    {'id': 'call_eS3k3cRgJzCjIdQH5Q1HsSsB',
     'function': {'arguments': '{"energetic": true, "loud": true, "bpm": "120"}',
      'name': 'start_music'},
     'type': 'function'},
    {'id': 'cal

Gemini can follow as well

In [64]:
prompt = "Nice. So please go back to the initial bpm."

response = gemini.run(prompt=prompt)

response.model_dump()

{'message': {'role': 'assistant',
  'content': [{'text': "Alright, I've switched the music back to the initial 120 BPM. It's still relaxed and at a lower volume, but with a slightly more upbeat tempo now. 🎶 \n\nLet me know if you want to make any other adjustments to the music or anything else in the room! 😊 \n"}],
  'name': None},
 'usage': {'model_name': 'gemini-1.5-pro',
  'prompt_tokens': 1330,
  'completion_tokens': 89},
 'prompt': [parts {
    text: "Turn this place into a comfortable party mood!"
  }
  role: "user",
  parts {
    function_call {
      name: "power_disco_ball"
      args {
        fields {
          key: "power"
          value {
            bool_value: true
          }
        }
      }
    }
  }
  parts {
    function_call {
      name: "start_music"
      args {
        fields {
          key: "loud"
          value {
            bool_value: true
          }
        }
        fields {
          key: "energetic"
          value {
            bool_value: true
  

In [65]:
prompt = "OK, then turn brightness to 0.7."

response = claude.run(prompt=prompt)

response.model_dump()

{'message': {'role': 'assistant',
  'content': [{'text': "Great! I've increased the brightness of the lights to 0.7, which should create a slightly brighter and more vibrant atmosphere while still maintaining a comfortable party mood. \n\nTo summarize the current setup:\n1. The disco ball is spinning\n2. The music is playing at 120 BPM, not too energetic or loud\n3. The lights are now at 70% brightness\n\nIs there anything else you'd like to adjust or change to perfect the ambiance?"}],
  'name': None},
 'usage': {'model_name': 'claude-3-5-sonnet-20240620',
  'prompt_tokens': 3349,
  'completion_tokens': 178},
 'prompt': [{'role': 'user',
   'content': [{'text': 'Turn this place into a comfortable party mood!',
     'type': 'text'}]},
  {'role': 'assistant',
   'content': [{'id': 'toolu_2meoHGpU9xnEskOBHYvOUVHa',
     'type': 'tool_use',
     'input': {'power': True},
     'name': 'power_disco_ball'},
    {'id': 'toolu_eS3k3cRgJzCjIdQH5Q1HsSsB',
     'type': 'tool_use',
     'input': {

In [66]:
prompt = "Finally stop the disco ball."

response = gemini.run(prompt)

response.model_dump()

{'message': {'role': 'assistant',
  'content': [{'text': 'Done! The disco ball has come to a stop. \n\nYour party space is now set with:\n\n* Relaxed music playing at 120 BPM\n* A comfortable brightness level\n* No disco ball spinning \n\nLet me know if you need anything else. Enjoy your party! 🎉 \n'}],
  'name': None},
 'usage': {'model_name': 'gemini-1.5-pro',
  'prompt_tokens': 1915,
  'completion_tokens': 76},
 'prompt': [parts {
    text: "Turn this place into a comfortable party mood!"
  }
  role: "user",
  parts {
    function_call {
      name: "power_disco_ball"
      args {
        fields {
          key: "power"
          value {
            bool_value: true
          }
        }
      }
    }
  }
  parts {
    function_call {
      name: "start_music"
      args {
        fields {
          key: "loud"
          value {
            bool_value: true
          }
        }
        fields {
          key: "energetic"
          value {
            bool_value: true
          }
  

If you input non-relavant topic, any tool is not called.

In [67]:
prompt = "What is the weather today?"

response = chat_openai.run(prompt)

response.model_dump()

{'message': {'role': 'assistant',
  'content': [{'text': "I'm unable to provide real-time weather updates or current data, including the weather for today. I recommend checking a reliable weather website or app for the most accurate and up-to-date information. If there's anything else I can assist you with, let me know!"}],
  'name': None},
 'usage': {'model_name': 'gpt-4o-mini-2024-07-18',
  'prompt_tokens': 1829,
  'completion_tokens': 102},
 'prompt': [{'role': 'user',
   'content': [{'type': 'text',
     'text': 'Turn this place into a comfortable party mood!'}],
   'name': 'User'},
  {'content': None,
   'refusal': None,
   'role': 'assistant',
   'function_call': None,
   'tool_calls': [{'id': 'call_2meoHGpU9xnEskOBHYvOUVHa',
     'function': {'arguments': '{"power": true}', 'name': 'power_disco_ball'},
     'type': 'function'},
    {'id': 'call_eS3k3cRgJzCjIdQH5Q1HsSsB',
     'function': {'arguments': '{"energetic": true, "loud": true, "bpm": "120"}',
      'name': 'start_music'

Conversation is recorded with serialized universal message format.

In [68]:
shared_memory.load()

[{'role': 'user',
  'content': [{'text': 'Turn this place into a comfortable party mood!'}],
  'name': None},
 {'role': 'function_call',
  'content': [{'name': 'power_disco_ball',
    'args': '{"power": true}',
    'call_id': 'call_2meoHGpU9xnEskOBHYvOUVHa'},
   {'name': 'start_music',
    'args': '{"energetic": true, "loud": true, "bpm": "120"}',
    'call_id': 'call_eS3k3cRgJzCjIdQH5Q1HsSsB'},
   {'name': 'dim_lights',
    'args': '{"brightness": 0.5}',
    'call_id': 'call_coFf4aSVXyk4M4XRmTuswYZo'}],
  'name': None},
 {'role': 'function',
  'content': [{'output': 'Disco ball is spinning!',
    'call_id': 'call_2meoHGpU9xnEskOBHYvOUVHa',
    'args': '{"power": true}',
    'funcname': 'power_disco_ball'},
   {'output': "Starting music! energetic=True loud=True, bpm='120'",
    'call_id': 'call_eS3k3cRgJzCjIdQH5Q1HsSsB',
    'args': '{"energetic": true, "loud": true, "bpm": "120"}',
    'funcname': 'start_music'},
   {'output': 'Lights are now set to 0.5',
    'call_id': 'call_coFf4aS

# Multi-turn conversation with tools and context truncation

Although context truncation doesn't work accurately if we use tools, but truncation is useful for token management to some extent.

In [69]:
from langrila import InMemoryConversationMemory

chat_openai = OpenAIFunctionalChat(
    api_key_env_name="API_KEY",
    model_name="gpt-4o-mini-2024-07-18",
    tools=[power_disco_ball, start_music, dim_lights],
    tool_configs=tool_configs,
    conversation_memory=InMemoryConversationMemory(),
    context_length=50,
)

In [70]:
prompt = "Turn this place into a comfortable party mood!"

response = chat_openai.run(prompt, tool_choice="auto")

In [71]:
prompt = "Can you make bpm up more?"

response = chat_openai.run(prompt=prompt, tool_choice="auto")

Input message is truncated because total length of messages exceeds context length.
Input message is truncated because total length of messages exceeds context length.


User message and assistant message are target of truncation while tool call message and tool result message is not.

In [72]:
print(response.message.content[0].text)

The music BPM has been increased to 180! Let's crank up the energy! 🎶💃


In [73]:
response.usage

Usage(prompt_tokens=529, completion_tokens=46, total_tokens=575)

In [74]:
response.prompt

[{'content': None,
  'refusal': None,
  'role': 'assistant',
  'function_call': None,
  'tool_calls': [{'id': 'call_tozi3YAFKRBvILnFiO3AzWdh',
    'function': {'arguments': '{"power": true}', 'name': 'power_disco_ball'},
    'type': 'function'},
   {'id': 'call_bNKKySVyrq4R2Y2ULczA6TOl',
    'function': {'arguments': '{"energetic": true, "loud": true, "bpm": "120"}',
     'name': 'start_music'},
    'type': 'function'},
   {'id': 'call_JRaW6ADcZzvdRlEUsm6gvX5o',
    'function': {'arguments': '{"brightness": 0.5}', 'name': 'dim_lights'},
    'type': 'function'}]},
 {'role': 'tool',
  'tool_call_id': 'call_tozi3YAFKRBvILnFiO3AzWdh',
  'name': 'power_disco_ball',
  'content': 'Disco ball is spinning!'},
 {'role': 'tool',
  'tool_call_id': 'call_bNKKySVyrq4R2Y2ULczA6TOl',
  'name': 'start_music',
  'content': "Starting music! energetic=True loud=True, bpm='120'"},
 {'role': 'tool',
  'tool_call_id': 'call_JRaW6ADcZzvdRlEUsm6gvX5o',
  'name': 'dim_lights',
  'content': 'Lights are now set t

In [75]:
# whole history
chat_openai.conversation_memory.load()

[{'role': 'user',
  'content': [{'text': 'Turn this place into a comfortable party mood!'}],
  'name': None},
 {'role': 'function_call',
  'content': [{'name': 'power_disco_ball',
    'args': '{"power": true}',
    'call_id': 'call_tozi3YAFKRBvILnFiO3AzWdh'},
   {'name': 'start_music',
    'args': '{"energetic": true, "loud": true, "bpm": "120"}',
    'call_id': 'call_bNKKySVyrq4R2Y2ULczA6TOl'},
   {'name': 'dim_lights',
    'args': '{"brightness": 0.5}',
    'call_id': 'call_JRaW6ADcZzvdRlEUsm6gvX5o'}],
  'name': None},
 {'role': 'function',
  'content': [{'output': 'Disco ball is spinning!',
    'call_id': 'call_tozi3YAFKRBvILnFiO3AzWdh',
    'args': '{"power": true}',
    'funcname': 'power_disco_ball'},
   {'output': "Starting music! energetic=True loud=True, bpm='120'",
    'call_id': 'call_bNKKySVyrq4R2Y2ULczA6TOl',
    'args': '{"energetic": true, "loud": true, "bpm": "120"}',
    'funcname': 'start_music'},
   {'output': 'Lights are now set to 0.5',
    'call_id': 'call_JRaW6AD